In [16]:
import os
from collections import Counter
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
import pylab as pl

In [17]:
%matplotlib inline
sns.set(style="darkgrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [18]:
#import math
from sklearn.preprocessing import OneHotEncoder, Imputer
%matplotlib inline

In [19]:
import gc

In [20]:
s=os.getcwd()
print(s)
pd.set_option('display.max_columns', 10)

C:\Users\sugan


In [21]:
# Loading data set
items = pd.read_csv('items.csv')
shops = pd.read_csv('shops.csv')
cats = pd.read_csv('item_categories.csv')
sales_train = pd.read_csv('sales_train.csv')
sales_test = pd.read_csv('sales_test.csv')

In [22]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [23]:
import time
import sys
import gc
import pickle
sys.version_info

sys.version_info(major=3, minor=7, micro=1, releaselevel='final', serial=0)

In [24]:
from itertools import product
from sklearn.preprocessing import LabelEncoder

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

In [26]:
sales_train = sales_train[sales_train.item_price<100000]
sales_train = sales_train[sales_train.item_cnt_day<1001]

In [27]:
median = sales_train[(sales_train.shop_id==32)&(sales_train.item_id==2973)&(sales_train.date_block_num==4)&(sales_train.item_price>0)].item_price.median()
sales_train.loc[sales_train.item_price<0, 'item_price'] = median

In [28]:
# Several shops are duplicates of each other (according to its name). Fix train and test set.

# Якутск Орджоникидзе, 56
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
sales_test.loc[sales_test.shop_id == 0, 'shop_id'] = 57

# Якутск ТЦ "Центральный"
sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
sales_test.loc[sales_test.shop_id == 1, 'shop_id'] = 58

# Жуковский ул. Чкалова 39м²
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
sales_test.loc[sales_test.shop_id == 10, 'shop_id'] = 11

In [29]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code', 'subtype_code']]

items.drop(['item_name'], axis=1, inplace=True)

In [30]:
len(list(set(sales_test.item_id) - set(sales_test.item_id).intersection(set(sales_train.item_id)))), len(list(set(sales_test.item_id))), len(sales_test)

(363, 5100, 214200)

In [31]:
ts = time.time()
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train[sales_train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)
time.time() - ts

22.0181782245636

In [32]:
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']

In [33]:
ts = time.time()
group = sales_train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) # NB clip target here
                                .astype(np.float16))
time.time() - ts

10.24032473564148

In [34]:
# To use time tricks append test pairs to the matrix.
sales_test['date_block_num'] = 34
sales_test['date_block_num'] = sales_test['date_block_num'].astype(np.int8)
sales_test['shop_id'] = sales_test['shop_id'].astype(np.int8)
sales_test['item_id'] = sales_test['item_id'].astype(np.int16)

In [35]:
ts = time.time()
matrix = pd.concat([matrix, sales_test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True) # 34 month
time.time() - ts

0.3526031970977783

In [36]:
# Shops/Items/Cats features

ts = time.time()
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items, on=['item_id'], how='left')
matrix = pd.merge(matrix, cats, on=['item_category_id'], how='left')
matrix['city_code'] = matrix['city_code'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['type_code'] = matrix['type_code'].astype(np.int8)
matrix['subtype_code'] = matrix['subtype_code'].astype(np.int8)
time.time() - ts

11.398956060409546

In [37]:
# Target Lags

def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [38]:
ts = time.time()
matrix = lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')
time.time() - ts

67.66717433929443

In [39]:
# Mean encoded features

ts = time.time()
group = matrix.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_avg_item_cnt')
matrix.drop(['date_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

22.393215656280518

In [40]:
ts = time.time()
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_cnt'] = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

83.28897905349731

In [41]:
ts = time.time()
group = matrix.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

84.63893628120422

In [42]:
ts = time.time()
group = matrix.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_category_id'], how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_cat_avg_item_cnt')
matrix.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

27.83671259880066

In [43]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,02.01.2013,0,59,22154,999.00,1.00,999.00
1,03.01.2013,0,25,2552,899.00,1.00,899.00
2,05.01.2013,0,25,2552,899.00,-1.00,-899.00
3,06.01.2013,0,25,2554,1709.05,1.00,1709.05
4,15.01.2013,0,25,2555,1099.00,1.00,1099.00


In [45]:
from sklearn.ensemble import RandomForestRegressor

In [47]:
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month,ID,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1
0,0,2,19,0.00,0.00,0,40,11,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,0,2,27,1.00,0.00,0,19,5,10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,0,2,28,0.00,0.00,0,30,8,55,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,0,2,29,0.00,0.00,0,23,5,16,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,0,2,32,0.00,0.00,0,40,11,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [53]:
matrix['year'] = matrix['date_block_num'].apply(lambda x: ((x//12) + 2013))
matrix['month'] = matrix['date_block_num'].apply(lambda x: (x % 12))

In [54]:
train_set = matrix.query('date_block_num >= 3 and date_block_num < 28').copy()
validation_set = matrix.query('date_block_num >= 28 and date_block_num < 33').copy()
test_set = matrix.query('date_block_num == 33').copy()

train_set.dropna(subset=['item_cnt_month'], inplace=True)
validation_set.dropna(subset=['item_cnt_month'], inplace=True)

train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)

In [55]:
X_train = train_set.drop(['item_cnt_month', 'date_block_num'], axis=1)
Y_train = train_set['item_cnt_month'].astype(int)
X_validation = validation_set.drop(['item_cnt_month', 'date_block_num'], axis=1)
Y_validation = validation_set['item_cnt_month'].astype(int)

In [56]:
int_features = ['shop_id', 'item_id', 'year', 'month']

X_train[int_features] = X_train[int_features].astype('int32')
X_validation[int_features] = X_validation[int_features].astype('int32')

In [57]:
latest_records = pd.concat([train_set, validation_set]).drop_duplicates(subset=['shop_id', 'item_id'], keep='last')
X_test = pd.merge(sales_test, latest_records, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])
X_test['year'] = 2015
X_test['month'] = 9
X_test.drop('item_cnt_month', axis=1, inplace=True)
X_test[int_features] = X_test[int_features].astype('int32')
X_test = X_test[X_train.columns]

In [58]:
sets = [X_train, X_validation, X_test]
           
for dataset in sets:
    for shop_id in dataset['shop_id'].unique():
        for column in dataset.columns:
            shop_median = dataset[(dataset['shop_id'] == shop_id)][column].median()
            dataset.loc[(dataset[column].isnull()) & (dataset['shop_id'] == shop_id), column] = shop_median
            
X_test.fillna(X_test.mean(), inplace=True)

In [59]:
print(X_test.head().T)

                                    0       1       2       3       4
shop_id                          5.00    5.00    5.00    5.00    5.00
item_id                       5037.00 5320.00 5233.00 5232.00 5268.00
ID                               0.00    1.00    2.00    3.00    4.00
city_code                        3.00    3.00    3.00    3.00    3.00
item_category_id                19.00   45.00   45.00   45.00   45.00
type_code                        5.00   12.00   12.00   12.00   12.00
subtype_code                    10.00   10.00   10.00   10.00   10.00
item_cnt_month_lag_1             3.00    0.00    0.00    0.00    0.00
item_cnt_month_lag_2             1.00    0.00    0.00    0.00    0.00
item_cnt_month_lag_3             1.00    0.00    0.00    0.00    0.00
item_cnt_month_lag_6             0.00    0.00    0.00    0.00    0.00
item_cnt_month_lag_12            1.00    0.00    0.00    0.00    0.00
date_avg_item_cnt_lag_1          0.29    0.29    0.29    0.29    0.29
date_item_avg_item_c

In [63]:
print(X_train.head().T)

                               4488710  4488713  4488714  4488715  4488719
shop_id                           2.00     2.00     2.00     2.00     2.00
item_id                          27.00    32.00    33.00    34.00    40.00
ID                                0.00     0.00     0.00     0.00     0.00
city_code                         0.00     0.00     0.00     0.00     0.00
item_category_id                 19.00    40.00    37.00    40.00    57.00
type_code                         5.00    11.00    11.00    11.00    13.00
subtype_code                     10.00     4.00     1.00     4.00     8.00
item_cnt_month_lag_1              0.00     0.00     1.00     0.00     0.00
item_cnt_month_lag_2              0.00     0.00     2.00     0.00     0.00
item_cnt_month_lag_3              0.00     0.00     0.00     0.00     0.00
item_cnt_month_lag_6              0.00     0.00     0.00     0.00     0.00
item_cnt_month_lag_12             1.00     0.00     1.00     0.00     0.00
date_avg_item_cnt_lag_1  

In [66]:
rf_features = ['shop_id', 'item_id', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2','item_cnt_month_lag_3','item_cnt_month_lag_6','item_cnt_month_lag_12','date_avg_item_cnt_lag_1',
               'date_item_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_2','date_item_avg_item_cnt_lag_3','date_item_avg_item_cnt_lag_6','date_item_avg_item_cnt_lag_12',
               'date_shop_avg_item_cnt_lag_1','date_shop_avg_item_cnt_lag_2','date_shop_avg_item_cnt_lag_3','date_shop_avg_item_cnt_lag_6','date_shop_avg_item_cnt_lag_12',
               'date_cat_avg_item_cnt_lag_1','year', 'month']
rf_train = X_train[rf_features]
rf_val = X_validation[rf_features]
rf_test = X_test[rf_features]

In [67]:
rf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)
rf_model.fit(rf_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [68]:
rf_train_pred = rf_model.predict(rf_train)
rf_val_pred = rf_model.predict(rf_val)
rf_test_pred = rf_model.predict(rf_test)

In [69]:
from sklearn.metrics import mean_squared_error

In [70]:
print('Train RMSE:', np.sqrt(mean_squared_error(Y_train, rf_train_pred)))
print('Validation RMSE:', np.sqrt(mean_squared_error(Y_validation, rf_val_pred)))

Train RMSE: 0.7127065649038405
Validation RMSE: 0.63673468576244


In [71]:
prediction_df = pd.DataFrame(sales_test['ID'], columns=['ID'])
prediction_df['item_cnt_month'] = rf_test_pred
prediction_df.to_csv('pred_submission.csv', index=False)
prediction_df.head(10)

,ID,item_cnt_month
0,0,1.45
1,1,0.07
2,2,0.07
3,3,0.07
4,4,0.07
5,5,0.79
6,6,0.07
7,7,0.07
8,8,0.07
9,9,0.07
